# Introduction

This notebook demonstrates how Python and R can work together within a single analytical workflow. 
- Python → Data engineering, preprocessing, and classification modeling
- R → Statistical visualization and geospatial mapping

Requirements:
- Before running this notebook, make sure you have installed:
- Python (3.9+ recommended)
- R (latest version)

rpy2 (Python–R bridge library)
- pip install rpy2
- pip install xgboost

**Part 1: Python**

- Data loading
- Cleaning and preprocessing
- Classification model
- Model evaluation

In [65]:
import pandas as pd
import numpy as np 

df = pd.read_csv(r'C:\Users\estep\OneDrive\Documentos\Github_repo\ProjectsRstudio\Python_to_R\india_city_aqi_2015_2023.csv')
df.head()

,city,date,pm25,pm10,no2,so2,co,o3,aqi,aqi_category
0,Delhi,2015-01-01,99.868566,147.103280,49.715328,19.615149,0.729754,46.487946,103,Moderate
1,Delhi,2015-01-02,143.168513,208.517207,32.957884,14.712800,0.660975,43.014054,141,Moderate
2,Delhi,2015-01-03,89.678491,101.412886,14.126233,9.188562,0.496151,54.713710,82,Satisfactory
3,Delhi,2015-01-04,43.679037,65.432963,61.984732,10.871118,0.820258,28.628777,50,Good
4,Delhi,2015-01-05,58.224691,110.443143,22.735096,13.878490,0.619808,45.624594,69,Satisfactory


In [66]:
print ("Dataframe Shape: ", df.shape)
print ("Dataframe Info: ", df.info)
print ("Dataframe Describe: ", df.describe)

Dataframe Shape:  (32870, 10)
Dataframe Info:  <bound method DataFrame.info of             city        date        pm25        pm10        no2        so2  \
0          Delhi  2015-01-01   99.868566  147.103280  49.715328  19.615149   
1          Delhi  2015-01-02  143.168513  208.517207  32.957884  14.712800   
2          Delhi  2015-01-03   89.678491  101.412886  14.126233   9.188562   
3          Delhi  2015-01-04   43.679037   65.432963  61.984732  10.871118   
4          Delhi  2015-01-05   58.224691  110.443143  22.735096  13.878490   
...          ...         ...         ...         ...        ...        ...   
32865  Ahmedabad  2023-12-27  104.189767  125.996540  54.037235   9.458740   
32866  Ahmedabad  2023-12-28   78.296822  126.139390  21.102546  14.506215   
32867  Ahmedabad  2023-12-29  133.796319  174.428556  14.262469   9.569303   
32868  Ahmedabad  2023-12-30   34.293435   67.092247  43.458757  13.238309   
32869  Ahmedabad  2023-12-31   52.469690   82.606862  27.533347

In [67]:
df.isnull().sum().to_frame(name="Null Count")


,Null Count
city,0
date,0
pm25,0
pm10,0
no2,0
so2,0
co,0
o3,0
aqi,0
aqi_category,0


In [68]:
y = df["aqi_category"]
X = df[["pm25", "pm10", "no2", "so2", "co", "o3"]]


In [69]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)



In [70]:
# Model

model = XGBClassifier(
    objective="multi:softprob",
    num_class=len(le.classes_),
    eval_metric="mlogloss",
    random_state=42
)

model.fit(X_train, y_train)


# Predictions

y_pred = model.predict(X_test)


# Evaluation

labels = np.arange(len(le.classes_))

print("\nClassification Report:\n")
print(
    classification_report(
        y_test,
        y_pred,
        labels=labels,
        target_names=le.classes_,
        zero_division=0
    )
)

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

        Good       0.98      0.98      0.98       836
    Moderate       0.99      0.99      0.99      2287
        Poor       1.00      0.50      0.67         2
Satisfactory       0.99      0.99      0.99      3449

    accuracy                           0.99      6574
   macro avg       0.99      0.86      0.91      6574
weighted avg       0.99      0.99      0.99      6574


Confusion Matrix:

[[ 819    0    0   17]
 [   0 2260    0   27]
 [   0    1    1    0]
 [  13   15    0 3421]]


**Model Performance and Class Imbalance**

The model achieves an overall accuracy of 0.99, indicating strong predictive performance across most categories. However, a closer inspection reveals a significant class imbalance issue.

The "Poor" category contains only 2 observations in the test set. As a result, its recall (0.50) and F1-score (0.67) are unstable and not statistically reliable.

This imbalance impacts the macro average score, which treats all classes equally, while the weighted average remains high due to the dominance of larger categories such as "Satisfactory" and "Moderate".

Therefore, while the model performs well globally, the results for minority classes should be interpreted with caution.

To address this limitation and improve interpretability, a simplified three-category classification will be explored in the next section.

In [71]:
# Create a simplified 3-class target from aqi_category
mapping_3 = {
    "Good": "Low",
    "Satisfactory": "Low",
    "Moderate": "Medium",
    "Poor": "High"
}

df["aqi_simplified"] = df["aqi_category"].map(mapping_3)

# (Optional) sanity checks
print(df["aqi_simplified"].value_counts(dropna=False))
print("Rows with unmapped categories:", df["aqi_simplified"].isna().sum())


aqi_simplified
Low       21427
Medium    11433
High         10
Name: count, dtype: int64
Rows with unmapped categories: 0


In [72]:

# Features and New Target (3 classes)

X = df[["pm25", "pm10", "no2", "so2", "co", "o3"]]
y = df["aqi_simplified"]

# Encode Target

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)


# Train-Test Split (Stratified)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)



# Model (XGBoost Multiclass)

from xgboost import XGBClassifier

model = XGBClassifier(
    objective="multi:softprob",
    num_class=len(le.classes_),   # now should be 3
    eval_metric="mlogloss",
    random_state=42
)

model.fit(X_train, y_train)



# Predictions

y_pred = model.predict(X_test)


#  Evaluation

from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

labels = np.arange(len(le.classes_))

print("\nClassification Report (3 Classes):\n")
print(
    classification_report(
        y_test,
        y_pred,
        labels=labels,
        target_names=le.classes_,
        zero_division=0
    )
)

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))



Classification Report (3 Classes):

              precision    recall  f1-score   support

        High       0.00      0.00      0.00         2
         Low       1.00      0.99      1.00      4285
      Medium       0.99      0.99      0.99      2287

    accuracy                           0.99      6574
   macro avg       0.66      0.66      0.66      6574
weighted avg       0.99      0.99      0.99      6574


Confusion Matrix:

[[   0    0    2]
 [   0 4261   24]
 [   0   12 2275]]


**Model Evaluation — Three-Class Simplification**

After simplifying the target variable into three categories (Low, Medium, High), the model maintains an overall accuracy of 0.99.

Performance remains strong for the majority classes:

- Low: near-perfect precision and recall
- Medium: consistently high precision and recall

However, the High category still presents severe class imbalance, with only 2 observations in the test set. As a result, the model fails to correctly identify this class, leading to precision and recall values of 0.00.

This indicates that:

- The dataset is overwhelmingly dominated by Low and Medium pollution levels.
- The model learns dominant patterns effectively.

Minority class prediction remains unreliable due to insufficient representation.

Therefore, while the simplified classification improves interpretability, it does not resolve the fundamental imbalance in the data distribution.

For visualization purposes, the simplified categories remain useful. However, statistical conclusions regarding the "High" category must be interpreted with caution.

**Part 2 : R**

- Advanced visualization
- Geospatial mapping of India
- Statistical interpretation


Python can absotely generate geospatial maps and interactive visualizations. However, building clean, publication-ready maps often requires additional setup and multiple libraries, which can increase complexity when the main goal is storytelling.

For this project, Python is used for data preparation and classification, while R is used for geospatial visualization because its ecosystem (e.g., sf, ggplot2, and mapping utilities) makes it straightforward to produce clear and aesthetically consistent maps with minimal overhead.

This approach reinforces the main idea of the notebook: Python and R do not compete — they complement each other.

In [73]:
# 1) Categoría dominante (moda) por ciudad
city_mode = (
    df.groupby("city")["aqi_simplified"]
      .agg(lambda s: s.value_counts().idxmax())
      .reset_index()
      .rename(columns={"aqi_simplified": "dominant_category"})
)

# 2) Promedio de AQI por ciudad (para alternativa numérica)
city_aqi = (
    df.groupby("city")["aqi"]
      .mean()
      .reset_index()
      .rename(columns={"aqi": "avg_aqi"})
)

# 3) Unir
city_summary = city_mode.merge(city_aqi, on="city", how="left")
city_summary

,city,dominant_category,avg_aqi
0,Ahmedabad,Low,88.525099
1,Bengaluru,Low,88.386979
2,Chennai,Low,87.905689
3,Delhi,Low,88.275936
4,Hyderabad,Low,88.883785
5,Jaipur,Low,87.619410
6,Kolkata,Low,88.300578
7,Lucknow,Low,87.884393
8,Mumbai,Low,86.853362
9,Pune,Low,88.591421


In [139]:
import pandas as pd


df = pd.read_csv(r'C:\Users\estep\OneDrive\Documentos\Github_repo\ProjectsRstudio\Python_to_R\india_city_aqi_2015_2023.csv')


# import df again or you can make a clean copy
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["year"] = df["date"].dt.year

# Agregate city by years
city_year = (
    df.groupby(["city", "year"], as_index=False)
      .agg(avg_aqi=("aqi", "mean"))
)

city_year.head()

#r requires coordinates

coords = {
    "Delhi": (28.6139, 77.2090),
    "Mumbai": (19.0760, 72.8777),
    "Bengaluru": (12.9716, 77.5946),
    "Hyderabad": (17.3850, 78.4867),
    "Chennai": (13.0827, 80.2707),
    "Kolkata": (22.5726, 88.3639),
    "Ahmedabad": (23.0225, 72.5714),
    "Pune": (18.5204, 73.8567),
    "Jaipur": (26.9124, 75.7873),
    "Lucknow": (26.8467, 80.9462),
}

coords_df = (
    pd.DataFrame(coords, index=["lat", "lon"])
      .T.reset_index()
      .rename(columns={"index": "city"})
)

city_year_map = city_year.merge(coords_df, on="city", how="left")

print("Missing coords:", city_year_map[["lat","lon"]].isna().sum().to_dict())
city_year_map.head()


Missing coords: {'lat': 0, 'lon': 0}


,city,year,avg_aqi,lat,lon
0,Ahmedabad,2015,91.257534,23.0225,72.5714
1,Ahmedabad,2016,90.702186,23.0225,72.5714
2,Ahmedabad,2017,86.123288,23.0225,72.5714
3,Ahmedabad,2018,87.704110,23.0225,72.5714
4,Ahmedabad,2019,85.057534,23.0225,72.5714


In [140]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import ipywidgets as widgets
from IPython.display import display, clear_output

# converter cuty
with localconverter(ro.default_converter + pandas2ri.converter):
    ro.globalenv["city_year_map"] = city_year_map

# prepare libraries
ro.r("""
library(ggplot2)
library(sf)
library(rnaturalearth)
library(rnaturalearthdata)
library(grid)

india <- ne_countries(country = "India", returnclass = "sf")
city_year_map$year <- as.integer(city_year_map$year)
""")

# python function that request R to generate a PNG file per year for slider
def render_map_year(year: int, outpath="map_year.png"):
    ro.globalenv["yr"] = year
    ro.globalenv["outpath"] = outpath
    
    ro.r("""
    d <- subset(city_year_map, year == yr)
    d$city_abbr <- d$city
        d$city_abbr[d$city == "Ahmedabad"] <- "AHM"
        d$city_abbr[d$city == "Bengaluru"] <- "BLR"
        d$city_abbr[d$city == "Chennai"] <- "CHE"
        d$city_abbr[d$city == "Delhi"] <- "DEL"
        d$city_abbr[d$city == "Hyderabad"] <- "HYD"
        d$city_abbr[d$city == "Jaipur"] <- "JAI"
        d$city_abbr[d$city == "Kolkata"] <- "KOL"
        d$city_abbr[d$city == "Lucknow"] <- "LKO"
        d$city_abbr[d$city == "Mumbai"] <- "MUM"
        d$city_abbr[d$city == "Pune"] <- "PUN"
            

    p <- ggplot() +
      geom_sf(data = india, fill = "gray90", color = "gray50") +
      geom_point(data = d, aes(x = lon, y = lat, color = avg_aqi), size = 10) +
      geom_text( data = d,  aes(x = lon, y = lat, label = paste0(round(avg_aqi, 1)," ",city_abbr)),  nudge_y = 0.8,  size = 4, color = 'black', fontface = 'bold') +
      scale_color_viridis_c(option = "cividis") +
      coord_sf( xlim = c(68, 95),  ylim = c(6, 37),  expand = FALSE ) +
      labs(
        title = paste0("India AQI — City-Level Average (", yr, ")"),
        subtitle = "Average AQI by city for the selected year",
        x = "Longitude", y = "Latitude", color = "Avg AQI"
      ) +
      theme_minimal() +
      theme(
        panel.background = element_rect(fill = "white", color = NA),
        plot.background  = element_rect(fill = "white", color = NA),
        legend.background = element_rect(fill = "white"),
        legend.title = element_text(size = 16, face = "bold"),
        legend.text  = element_text(size = 14),
        legend.key.height = unit(1.2, "cm"),
        legend.key.width  = unit(0.6, "cm"),
        panel.grid.major = element_line(color = "gray85"),
        panel.grid.minor = element_blank()
      ) 
    ggsave(filename = outpath, plot = p, width = 10, height = 10, dpi = 350)
    """)
    return outpath

# slider
years = sorted(city_year_map["year"].dropna().unique().astype(int).tolist())
slider = widgets.IntSlider(value=years[0], min=min(years), max=max(years), step=1, description="Year:")

out = widgets.Output()

def on_change(change):
    if change["name"] == "value":
        with out:
            clear_output(wait=True)
            path = render_map_year(change["new"], outpath="map_year.png")
            img = Image.open(path)
            plt.figure(figsize=(9,6))
            plt.imshow(img)
            plt.axis("off")
            plt.show()

slider.observe(on_change)
display(slider, out)

# render i
on_change({"name":"value", "new": slider.value})


IntSlider(value=2015, description='Year:', max=2023, min=2015)

Output()

**Interactive AQI Map (R Visualization Integrated in Python)**

The interactive map reveals that air quality levels vary consistently across major Indian cities over time. While the overall average AQI values remain within a relatively narrow range, certain cities (such as Delhi and Ahmedabad) tend to exhibit higher average AQI levels compared to southern cities like Chennai and Bengaluru.

The visualization highlights spatial patterns more effectively than a traditional table or static plot. By allowing year-by-year exploration, the slider demonstrates how temporal variation can be examined dynamically, reinforcing the importance of interactive storytelling in environmental data analysis.

Although the AQI averages do not fluctuate dramatically between years, the map shows persistent spatial differences across cities. Northern urban centers generally present higher pollution averages, whereas some southern cities maintain comparatively lower values.

The interactive structure allows users to explore temporal trends intuitively, making spatial disparities more interpretable than through numeric summaries alone.

This final visualization reinforces the project’s core idea: Python excels in data engineering and modeling workflows, while R provides powerful and expressive visualization tools. Together, they create a more complete analytical pipeline.